# LangGraph Basics: Graph API vs Functional API

**Updated with Official LangGraph Functional API Documentation**

This notebook demonstrates LangGraph concepts using **both APIs**:
- **Graph API (StateGraph)**: Explicit, declarative graph construction
- **Functional API**: Imperative, uses `@entrypoint` and `@task` decorators

## Learning Objectives
1. Build the same workflows using both APIs
2. Understand when to use each approach
3. Compare code structure and complexity
4. Learn the strengths of each paradigm

## Labs Covered
1. **Lab 1**: Linear Workflow - Employee Onboarding
2. **Lab 2**: Conditional Routing - Leave Approval System

## Key Functional API Requirements
⚠️ **IMPORTANT**: 
- `@entrypoint` functions must accept **ONE positional argument** (use dict for multiple inputs)
- `@task` functions return futures - call `.result()` to get the value
- All inputs/outputs must be **JSON-serializable**

## Setup and Installation

In [ ]:
# Install required packages
!pip install --pre -qU langgraph langchain langchain-openai

In [ ]:
# Import required libraries
import os
from typing import TypedDict, Annotated, Literal
from langchain_openai import ChatOpenAI

# Configure API key
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("✅ Setup complete!")

---
# Lab 1: Employee Onboarding Workflow

**Scenario**: Create a linear workflow that onboards new employees through these steps:
1. Validate employee information
2. Assign equipment based on department
3. Set up accounts
4. Send welcome email

We'll build this **twice** - once with each API.

## Approach 1: Using Graph API (StateGraph)

### Step 1: Define State

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# Define state schema
class OnboardingState(TypedDict):
    """State for employee onboarding workflow"""
    employee_name: str
    employee_id: str
    department: str
    messages: Annotated[list, add_messages]
    validation_status: str
    tasks_completed: list

print("✅ Graph API: State defined")

### Step 2: Define Nodes (Functions)

In [ ]:
def validate_employee_graph(state: OnboardingState):
    """Node 1: Validate employee information"""
    print(f"📋 [Graph API] Validating {state['employee_name']}...")
    
    if state['employee_name'] and state['employee_id']:
        return {
            "validation_status": "✅ Validated",
            "messages": [("assistant", f"Employee {state['employee_name']} validated")]
        }
    return {"validation_status": "❌ Failed"}

def assign_equipment_graph(state: OnboardingState):
    """Node 2: Assign equipment"""
    print(f"💻 [Graph API] Assigning equipment for {state['department']}...")
    
    equipment = {
        "Engineering": ["MacBook Pro", "Monitor"],
        "HR": ["Laptop", "Headset"],
        "Sales": ["MacBook Air", "Phone"],
    }
    
    return {
        "tasks_completed": state['tasks_completed'] + ["Equipment assigned"],
        "messages": [("assistant", f"Assigned: {', '.join(equipment.get(state['department'], ['Laptop']))}")]
    }

def setup_accounts_graph(state: OnboardingState):
    """Node 3: Setup accounts"""
    print(f"🔐 [Graph API] Setting up accounts...")
    
    return {
        "tasks_completed": state['tasks_completed'] + ["Accounts created"],
        "messages": [("assistant", "Created: Email, Slack, HR Portal")]
    }

def send_welcome_graph(state: OnboardingState):
    """Node 4: Send welcome email"""
    print(f"📧 [Graph API] Sending welcome email...")
    
    return {
        "tasks_completed": state['tasks_completed'] + ["Welcome email sent"],
        "messages": [("assistant", f"Welcome email sent to {state['employee_name']}")]
    }

print("✅ Graph API: All nodes defined")

### Step 3: Build the Graph

In [ ]:
# Create graph
workflow_graph = StateGraph(OnboardingState)

# Add nodes
workflow_graph.add_node("validate", validate_employee_graph)
workflow_graph.add_node("assign_equipment", assign_equipment_graph)
workflow_graph.add_node("setup_accounts", setup_accounts_graph)
workflow_graph.add_node("send_welcome", send_welcome_graph)

# Add edges (define the flow)
workflow_graph.add_edge(START, "validate")
workflow_graph.add_edge("validate", "assign_equipment")
workflow_graph.add_edge("assign_equipment", "setup_accounts")
workflow_graph.add_edge("setup_accounts", "send_welcome")
workflow_graph.add_edge("send_welcome", END)

# Compile
app_graph = workflow_graph.compile()

print("✅ Graph API: Workflow compiled")
print("Flow: START → validate → assign → setup → welcome → END")

### Step 4: Run the Graph API Workflow

In [ ]:
print("\n" + "="*70)
print("GRAPH API: Running Onboarding Workflow")
print("="*70)

initial_state = {
    "employee_name": "Karan Singh",
    "employee_id": "106",
    "department": "Engineering",
    "messages": [],
    "validation_status": "",
    "tasks_completed": []
}

result_graph = app_graph.invoke(initial_state)

print("\n" + "="*70)
print("RESULT")
print("="*70)
print(f"Status: {result_graph['validation_status']}")
print(f"Tasks: {len(result_graph['tasks_completed'])}")
for task in result_graph['tasks_completed']:
    print(f"  ✓ {task}")

## Approach 2: Using Functional API

Now let's build the **same workflow** using the Functional API with `@entrypoint` and `@task`.

### ⚠️ Key Requirements:
1. **@entrypoint must accept ONE positional argument** (use dict for multiple inputs)
2. **Tasks return futures** - use `.result()` to get the value
3. **Use a checkpointer** for persistence features

In [ ]:
from langgraph.func import entrypoint, task
from langgraph.checkpoint.memory import MemorySaver

# Define tasks (these can run in parallel if needed)
@task()
def validate_employee_func(inputs: dict) -> dict:
    """Task 1: Validate employee"""
    print(f"📋 [Functional API] Validating {inputs['name']}...")
    
    if inputs['name'] and inputs['emp_id']:
        return {
            "status": "✅ Validated",
            "message": f"Employee {inputs['name']} validated"
        }
    return {"status": "❌ Failed"}

@task()
def assign_equipment_func(department: str) -> dict:
    """Task 2: Assign equipment"""
    print(f"💻 [Functional API] Assigning equipment for {department}...")
    
    equipment = {
        "Engineering": ["MacBook Pro", "Monitor"],
        "HR": ["Laptop", "Headset"],
        "Sales": ["MacBook Air", "Phone"],
    }
    
    return {"equipment": equipment.get(department, ["Laptop"])}

@task()
def setup_accounts_func(name: str) -> dict:
    """Task 3: Setup accounts"""
    print(f"🔐 [Functional API] Setting up accounts for {name}...")
    
    return {"accounts": ["Email", "Slack", "HR Portal"]}

@task()
def send_welcome_func(name: str) -> dict:
    """Task 4: Send welcome email"""
    print(f"📧 [Functional API] Sending welcome email...")
    
    return {"email_sent": True, "message": f"Welcome email sent to {name}"}

# Create checkpointer for persistence
checkpointer = MemorySaver()

# Define the entrypoint - MUST accept single positional argument!
@entrypoint(checkpointer=checkpointer)
def onboard_employee_func(inputs: dict) -> dict:
    """
    Main workflow using Functional API
    
    CRITICAL: This function accepts ONE dict argument containing all inputs
    """
    # Extract inputs
    employee_name = inputs["employee_name"]
    employee_id = inputs["employee_id"]
    department = inputs["department"]
    
    # Step 1: Validate (task returns a future, use .result() to get value)
    validation = validate_employee_func({
        "name": employee_name,
        "emp_id": employee_id
    }).result()
    
    if validation["status"] == "❌ Failed":
        return {"error": "Validation failed"}
    
    # Step 2: Assign equipment
    equipment = assign_equipment_func(department).result()
    
    # Step 3: Setup accounts
    accounts = setup_accounts_func(employee_name).result()
    
    # Step 4: Send welcome
    welcome = send_welcome_func(employee_name).result()
    
    # Return final result (must be JSON-serializable)
    return {
        "validation_status": validation["status"],
        "equipment_assigned": equipment["equipment"],
        "accounts_created": accounts["accounts"],
        "welcome_sent": welcome["email_sent"],
        "tasks_completed": 4
    }

print("✅ Functional API: Workflow defined")

### Run the Functional API Workflow

In [ ]:
print("\n" + "="*70)
print("FUNCTIONAL API: Running Onboarding Workflow")
print("="*70)

# Create config with thread_id for persistence
config = {"configurable": {"thread_id": "onboarding-123"}}

# Invoke the workflow - pass inputs as a SINGLE dict
result_func = onboard_employee_func.invoke(
    {
        "employee_name": "Karan Singh",
        "employee_id": "106",
        "department": "Engineering"
    },
    config=config
)

print("\n" + "="*70)
print("RESULT")
print("="*70)
print(f"Status: {result_func['validation_status']}")
print(f"Equipment: {', '.join(result_func['equipment_assigned'])}")
print(f"Accounts: {', '.join(result_func['accounts_created'])}")
print(f"Tasks completed: {result_func['tasks_completed']}")

## 🔍 Lab 1 Comparison

### Graph API
**Pros:**
- ✅ Explicit visualization of workflow
- ✅ Easy to modify flow by changing edges
- ✅ Better for complex multi-actor systems
- ✅ State transitions are explicit
- ✅ More granular checkpointing (after each node)

**Cons:**
- ❌ More boilerplate code
- ❌ Need to define State schema
- ❌ More verbose for simple workflows

### Functional API
**Pros:**
- ✅ Less boilerplate, cleaner code
- ✅ Familiar Python programming style
- ✅ Easier for simple linear workflows
- ✅ No explicit state management needed
- ✅ Automatic checkpointing of task results

**Cons:**
- ❌ Flow is implicit in code
- ❌ No visualization support
- ❌ Single dict input requirement can be awkward
- ❌ Less granular checkpointing (at entrypoint level)

### When to Use Which?
- **Graph API**: Complex workflows, multi-agent systems, need visualization, granular control
- **Functional API**: Simple workflows, rapid prototyping, prefer imperative style, existing codebases

---
# Lab 2: Conditional Routing - Leave Approval

**Scenario**: Route leave requests based on conditions:
- Insufficient balance → Reject
- ≤3 days + sufficient balance → Auto-approve
- >3 days + sufficient balance → Manager review

This demonstrates **conditional edges** (Graph API) vs **if/else** (Functional API).

## Setup: Employee Database

⚠️ **IMPORTANT**: This cell MUST be run before the Functional API cells below!

In [ ]:
# Employee data - MUST BE DEFINED BEFORE FUNCTIONAL API TASKS!
employees = {
    "101": {"name": "Priya Sharma", "department": "Engineering", "manager": "102"},
    "102": {"name": "Rahul Verma", "department": "Engineering", "manager": "103"},
    "103": {"name": "Anjali Patel", "department": "HR", "manager": None},
}

leave_balances = {"101": 12, "102": 8, "103": 15}

print("✅ Employee database loaded")
print(f"Leave balances: {leave_balances}")

## Approach 1: Graph API with Conditional Edges

In [ ]:
# Define state
class LeaveState(TypedDict):
    employee_id: str
    employee_name: str
    days_requested: int
    leave_balance: int
    approval_status: str
    messages: Annotated[list, add_messages]

# Define nodes
def check_balance_graph(state: LeaveState):
    print(f"\n📊 [Graph API] Checking balance for {state['employee_name']}...")
    balance = leave_balances.get(state['employee_id'], 0)
    return {"leave_balance": balance}

def auto_approve_graph(state: LeaveState):
    print(f"✅ [Graph API] Auto-approved!")
    return {
        "approval_status": "✅ Auto-Approved",
        "messages": [("assistant", "Leave approved automatically")]
    }

def manager_review_graph(state: LeaveState):
    print(f"👔 [Graph API] Sent to manager...")
    return {
        "approval_status": "⏳ Pending Manager Review",
        "messages": [("assistant", "Sent to manager for approval")]
    }

def reject_graph(state: LeaveState):
    print(f"❌ [Graph API] Rejected!")
    return {
        "approval_status": "❌ Rejected - Insufficient Balance",
        "messages": [("assistant", "Request denied due to insufficient balance")]
    }

# Routing function - KEY CONCEPT!
def route_leave_graph(state: LeaveState) -> Literal["approve", "manager", "reject"]:
    """
    This function decides which node to go to next!
    """
    days = state['days_requested']
    balance = state['leave_balance']
    
    # Check balance first
    if days > balance:
        return "reject"
    
    # Short leaves auto-approved
    if days <= 3:
        return "approve"
    
    # Long leaves need manager approval
    return "manager"

# Build graph
leave_workflow = StateGraph(LeaveState)

# Add nodes
leave_workflow.add_node("check_balance", check_balance_graph)
leave_workflow.add_node("approve", auto_approve_graph)
leave_workflow.add_node("manager", manager_review_graph)
leave_workflow.add_node("reject", reject_graph)

# Add edges
leave_workflow.add_edge(START, "check_balance")

# Add CONDITIONAL edge - this is the key!
leave_workflow.add_conditional_edges(
    "check_balance",
    route_leave_graph,
    {
        "approve": "approve",
        "manager": "manager",
        "reject": "reject"
    }
)

# All routes end
leave_workflow.add_edge("approve", END)
leave_workflow.add_edge("manager", END)
leave_workflow.add_edge("reject", END)

# Compile
leave_app_graph = leave_workflow.compile()

print("✅ Graph API: Conditional workflow compiled")

### Test Graph API - Different Scenarios

In [ ]:
test_cases_graph = [
    {"employee_id": "101", "employee_name": "Priya", "days_requested": 2},  # Auto-approve
    {"employee_id": "102", "employee_name": "Rahul", "days_requested": 5}, # Manager review
    {"employee_id": "103", "employee_name": "Anjali", "days_requested": 20}, # Reject
]

print("\n" + "="*70)
print("GRAPH API: Testing Leave Approval")
print("="*70)

for test in test_cases_graph:
    initial = {**test, "leave_balance": 0, "approval_status": "", "messages": []}
    result = leave_app_graph.invoke(initial)
    print(f"\n{result['employee_name']} ({result['days_requested']} days): {result['approval_status']}")

## Approach 2: Functional API with If/Else

⚠️ **MAKE SURE** you ran the "Employee Database" cell above first!

In [ ]:
# Verify leave_balances is defined
try:
    print(f"✅ leave_balances is accessible: {leave_balances}")
except NameError:
    print("❌ ERROR: leave_balances not defined! Run the 'Employee Database' cell above first.")
    raise

In [ ]:
from langgraph.func import entrypoint, task
from langgraph.checkpoint.memory import MemorySaver

# Tasks - they will use the leave_balances defined above
@task()
def check_balance_func(employee_id: str) -> int:
    """Check employee leave balance"""
    print(f"📊 [Functional API] Checking balance...")
    # Access the global leave_balances dictionary
    return leave_balances.get(employee_id, 0)

@task()
def auto_approve_func() -> str:
    """Auto-approve leave"""
    print(f"✅ [Functional API] Auto-approved!")
    return "✅ Auto-Approved"

@task()
def manager_review_func() -> str:
    """Send to manager for review"""
    print(f"👔 [Functional API] Sent to manager...")
    return "⏳ Pending Manager Review"

@task()
def reject_func() -> str:
    """Reject leave request"""
    print(f"❌ [Functional API] Rejected!")
    return "❌ Rejected - Insufficient Balance"

# Create checkpointer
leave_checkpointer = MemorySaver()

# Entrypoint with standard if/else logic - MUST accept single dict!
@entrypoint(checkpointer=leave_checkpointer)
def approve_leave_func(inputs: dict) -> dict:
    """
    Leave approval using Functional API
    
    Uses standard Python if/else - no routing function needed!
    MUST accept single dict argument.
    """
    # Extract inputs
    employee_id = inputs["employee_id"]
    employee_name = inputs["employee_name"]
    days_requested = inputs["days_requested"]
    
    # Check balance (returns a future)
    balance = check_balance_func(employee_id).result()
    
    # Routing logic with standard Python if/else
    if days_requested > balance:
        status = reject_func().result()
    elif days_requested <= 3:
        status = auto_approve_func().result()
    else:
        status = manager_review_func().result()
    
    # Return result (must be JSON-serializable)
    return {
        "employee_name": employee_name,
        "days_requested": days_requested,
        "balance": balance,
        "approval_status": status
    }

print("✅ Functional API: Leave approval workflow defined")

### Test Functional API - Same Scenarios

In [ ]:
test_cases_func = [
    {"employee_id": "101", "employee_name": "Priya", "days_requested": 2},
    {"employee_id": "102", "employee_name": "Rahul", "days_requested": 5},
    {"employee_id": "103", "employee_name": "Anjali", "days_requested": 20},
]

print("\n" + "="*70)
print("FUNCTIONAL API: Testing Leave Approval")
print("="*70)

for i, test in enumerate(test_cases_func):
    # Each test needs unique thread_id for proper checkpointing
    config = {"configurable": {"thread_id": f"leave-test-{i}"}}
    result = approve_leave_func.invoke(test, config=config)
    print(f"\n{result['employee_name']} ({result['days_requested']} days): {result['approval_status']}")

## 🔍 Lab 2 Comparison

### Conditional Logic

**Graph API:**
```python
# Routing function returns node name
def route(state) -> Literal["a", "b", "c"]:
    if condition1:
        return "a"
    return "b"

# Add conditional edge
graph.add_conditional_edges("node", route, {"a": "node_a", "b": "node_b"})
```

**Functional API:**
```python
@entrypoint()
def workflow(inputs: dict):
    if condition1:
        result = task_a().result()
    else:
        result = task_b().result()
    return result
```

### Which is Better?

**Graph API wins when:**
- Complex branching with many paths
- Need to visualize decision flow
- Multiple conditional routes from same node
- Team needs to understand flow at a glance

**Functional API wins when:**
- Simple if/else logic
- Prefer standard Python patterns
- Want less boilerplate
- Rapid prototyping

## 📊 Final Comparison Summary

| Aspect | Graph API | Functional API |
|--------|-----------|----------------|
| **Paradigm** | Declarative | Imperative |
| **State Management** | Explicit TypedDict | Function-scoped |
| **Control Flow** | Nodes + Edges | Standard Python |
| **Visualization** | ✅ Easy | ❌ Not supported |
| **Code Lines** | More | Less |
| **Learning Curve** | Steeper | Gentler |
| **Best For** | Complex systems | Simple workflows |
| **Checkpointing** | After each node | After entrypoint + tasks |
| **Input Format** | Flexible | Single dict only |

## 🎓 Key Takeaways

1. **Both APIs are powerful** - choose based on your needs
2. **Graph API excels** at complex, multi-actor systems with visualization needs
3. **Functional API excels** at rapid development and simple flows
4. **@entrypoint requires single dict input** - this is a key constraint
5. **Tasks return futures** - always call `.result()` to get the value
6. **Use checkpointer** for persistence and human-in-the-loop features
7. **Global variables in tasks** - Make sure data is defined before task definitions

## ⚠️ Common Pitfalls

### Functional API:
- ❌ Passing multiple parameters to `@entrypoint` (must be single dict)
- ❌ Forgetting to call `.result()` on task futures
- ❌ Not providing a `checkpointer` (needed for persistence)
- ❌ Returning non-JSON-serializable values
- ❌ **Using undefined variables in tasks** (like leave_balances) - define them first!

### Graph API:
- ❌ Forgetting to compile the graph
- ❌ Not defining state schema properly
- ❌ Incorrect routing function return types

## 🚀 Next Steps

- Try adding human-in-the-loop with `interrupt()`
- Explore parallel task execution
- Learn about persistence and memory
- Build multi-agent systems with Graph API
- Integrate with existing codebases using Functional API